    HOT WIRE ANEMOMETER AQUISITON Z v.3
    LEA - Wind Tunnel
    python version

Developer: Rafael C. F. Mendes

Date: 04/10/2023   (v1: 24/11/2016)

Description: Use this program to move hot wire probe and taking data from CTA module.
             
Hardware: 
   - Arduino Uno (probe positioning system) with GRBL library (see more at https://github.com/grbl/grbl)
   - National Instrument - USB-4332 **Warning: Working voltage -10 to 10V**                 
   - Hot wire anemometer - miniCTA DANTEC

Before run it, install the following libraries:

    - pip install nidaqmx
    - pip install pyserial

In [3]:
#libraries

import nidaqmx          
import time
import numpy as np
import serial           
from datetime import datetime
from nidaqmx.constants import LineGrouping
from nidaqmx.constants import Edge
from nidaqmx.constants import AcquisitionType 
import os
from scipy.optimize import curve_fit


In [4]:
#Sampling Definitions 
SampleRate=1000
AquisitionTimeInSeconds=5

dz=10 # delta Y in mm
y = 310 # y coordinate to measure the profile; y=310 ==> middle 

# Define the serial port and baud rate
port = 'COM7'  # Update with your Arduino's port
baud_rate = 115200

In [17]:
#
z = np.arange(0, 331, dz)

Ysize = 1
Zsize = np.shape(z)



In [95]:
# Function to call NI system 

def acquire_data(SampleRate, AquisitionTimeInSeconds):
    with nidaqmx.Task() as master_task:
        Sample=SampleRate*AquisitionTimeInSeconds
        master = master_task.ai_channels.add_ai_voltage_chan("Dev1/ai0")
        master_task.timing.cfg_samp_clk_timing(SampleRate, source='', active_edge=Edge.RISING, sample_mode=AcquisitionType.CONTINUOUS)
        master_task.stop()
        master_data = master_task.read(Sample, timeout=AquisitionTimeInSeconds+1.0)
    return master_data


In [ ]:
# Create a serial object
ser = serial.Serial(port, baud_rate)

# Check if the serial port is open
if ser.is_open:
    print(f"Serial port {port} is open. Ready to communicate.")

ser.reset_input_buffer()
ser.reset_output_buffer()
ser.readline()

# Read data from Arduino
response = ser.readline().decode().strip()  # Read the response from Arduino
print("Response from Arduino:", response)

# Read data from Arduino
response = ser.readline().decode().strip()  # Read the response from Arduino
print("Response from Arduino:", response)

# Close the serial port
#ser.close()

# Send a command to Arduino
command = '$H\r\n'  # Example command
ser.write(command.encode())  # Encode the command and send it



In [99]:

folder_name = "Velocity data"

if not os.path.exists(folder_name):
    # Create the folder
    os.mkdir(folder_name)

now = datetime.now()
date_string = now.strftime('%d-%b-%Y')
time_string = now.strftime('%H-%M-%S')

filename = os.path.join(folder_name,f"{date_string}_{time_string}.txt")


In [ ]:
i = 1

command = 'z-100\r\n'  # Example command
ser.write(command.encode())  # Encode the command and send it

response = ser.readline().decode().strip()  # Read the response from Arduino
print("Response from Arduino:", response)
ser.close()

fileprint = f"{y[i]},0"
# Assuming 's' represents a file, open it for writing
with open('your_file.txt', 'w') as s:
    s.write(fileprint)
time.sleep(20)  # Sleep for 20 seconds

In [29]:
i = 1
j=1

fileprint = f"{y},{z[j]}"  # Use y and z directly, no need for indexing
print(fileprint)

# Introduce a 20-second delay
time.sleep(20)  # Pause for 20 seconds



310,10


In [ ]:


for i in range(n):
    for j in range(m):
        fileprint = f"{y[i]},{z[j]}"
        # Assuming 's' represents a file, open it for writing
        with open('your_file.txt', 'w') as s:
            s.write(fileprint)
        time.sleep(30)  # Sleep for 30 seconds

        folder_name = "your_folder_name"
        filename = os.path.join(folder_name, f"{y[i]}x{z[j]}.txt")

        data = Ni_device.startForeground()
        in, im = data.shape

        with open(filename, 'wt') as fid:
            for k in range(in):
                fid.write(f'{data[k]:12.8f}\n')

    with open(filename, 'wt') as fid:
        data = Ni_device.startForeground()
        in, im = data.shape
        for k in range(in):
            fid.write(f'{data[k]:12.8f}\n')


Serial port COM7 is open. Ready to communicate.
Response from Arduino: Grbl 1.1f ['$' for help]
Response from Arduino: [MSG:'$H'|'$X' to unlock]
